# **<center> Análisis de Sentimientos de los candidatos presidenciales </br> Ecuador 2021, en la red social Twitter </center>**
<h6>
    <center> Miguel Angel Macias, Jonnathan Campoberde <br/> 
    <i>Universidad de Cuenca <br/>  <i>Facultad de Ingenieria - Escuela de Sistemas<br/> Cuenca, Ecuador</i>
    <br/> { mangel.maciasn, jonnathan.campoberde }@ucuenca.edu.ec</center>
<h6>

# **I.&emsp;Introducción**
El presente trabajo tiene como objetivo emplear de manera práctica todos los conocimientos adquiridos en el transcurso de la asignatura “Text Mining”. En este trabajo se realizó una limpieza de los datos, se removieron caracteres especiales, emojis y enlaces de los tweets recolectados; así también se realizó una inspección de posibles cuentas trolls que afecten el AS de las opiniones de usuarios a favor o en contra de un movimiento político. Luego del tratamiento y análisis de los tweets se entrenaron 3 modelos de clasificación de texto usando Support Vector Machine (SVM), Naive Bayes (NB) y Random Forest (RF) para posteriormente compararlos y elegir qué modelo presentó los mejores resultados. 
Los tweets que se recolectaron tratan acerca de los dos candidatos presidenciales que lideran las encuestas para las elecciones de Ecuador 2021. Los dos candidatos, Andrés Arauz y Guillermo Lasso, pertenecen a los dos movimientos políticos más dominantes del estado ecuatoriano.

## **1.1.&emsp;Objetivos del Proyecto**
>. El objetivo de este artículo es el AS de estos dos candidatos para luego contrastarlos con los resultados oficiales de las elecciones, así también, entrenar un modelo de clasificación de sentimientos de los tweets que se generen en redes sociales acerca de la política dentro del país. Posteriormente se desea responder a las siguientes preguntas de investigación definidas en 1.2.

## **1.2.&emsp;Preguntas de investigación**
>- RQ1:     ¿Qué tan preciso los datos de Twitter reflejan el sentimiento político en un proceso electoral?
>- RQ1:     ¿En qué grado de aceptación se encuentran los candidatos dentro de la red social Twitter?
>- RQ3:     ¿Los resultados oficiales son parecidos a los resultados encontrados en nuestro análisis? 


In [54]:
import pandas as pd
import numpy as np
import stanza
import warnings
warnings.filterwarnings("ignore")

In [55]:
#!pip install stanza
#import stanza
#stanza.download('es')

# Text Cleaninng the data to the traininig the model

In [2]:
data_processed = '../data/processed/'  # directorio que contiene data procesada

## Arauz

In [4]:
df_arauz_tag = pd.read_csv(data_processed+'arauz_tag.csv')
df_arauz_tag.head(2)

,created_at,favorite_count,full_text,id_str,metadata.iso_language_code,retweeted_status.created_at,retweeted_status.full_text,user.created_at,user.favourites_count,user.followers_count,user.friends_count,user.id_str,user.location,user.verified,retweet_count,is_troll,sentiment_tag,sentiment_sc
0,2021-01-24 11:09:43+00:00,0,el lelo andrés arauz nos ha dicho q ha gestion...,1353298931913019393,es,Sat Jan 23 22:05:45 +0000 2021,El LELO Andrés Arauz nos ha dicho q ha gestion...,2019-07-08 04:50:16+00:00,23792,208,814,1148091911183503360,NaN,False,212,False,Negativo,0.072007
1,2021-01-24 11:11:10+00:00,0,a qué edad se enteraron que jorge glas y andré...,1353299298407096320,es,Sat Jan 23 23:38:46 +0000 2021,A qué edad se enteraron que Jorge Glas y André...,2012-10-09 14:57:55+00:00,1835,88,325,870030324,NaN,False,545,False,Negativo,0.311006


## Lasso

In [6]:
df_lasso_tag = pd.read_csv(data_processed+'lasso_tag.csv')
df_lasso_tag.head(2)

,created_at,favorite_count,full_text,id_str,metadata.iso_language_code,retweeted_status.created_at,retweeted_status.full_text,user.created_at,user.favourites_count,user.followers_count,user.friends_count,user.id_str,user.location,user.verified,retweet_count,is_troll,sentiment_tag,sentiment_sc
0,2021-01-24 18:48:13+00:00,0,se imaginan un gobierno de guillermo lasso con...,1353414316947550213,es,Sat Jan 23 23:24:31 +0000 2021,Se imaginan un gobierno de Guillermo Lasso con...,2020-01-05 21:38:08+00:00,22225,150,219,1213937693018607616,"Quito, Ecuador",False,409,False,Negativo,0.220347
1,2021-01-24 18:50:06+00:00,0,lenín moreno no gobernó con alianza pais lo hi...,1353414791684042757,es,Mon Jan 18 22:50:08 +0000 2021,"Lenín Moreno NO gobernó con Alianza PAIS, lo h...",2018-10-23 14:39:27+00:00,3027,228,1384,1054744113814429696,NaN,False,9,False,Positivo,0.741087


## Join Data to train the modelos

In [10]:
df_all_tweets = data_tweets_arauz = pd.concat([df_arauz_tag, df_lasso_tag], axis=0)
df_all_tweets = df_all_tweets.reset_index(drop=True)
df_all_tweets = data_tweets_arauz.sort_values('created_at')
df_all_tweets.head(2)

,created_at,favorite_count,full_text,id_str,metadata.iso_language_code,retweeted_status.created_at,retweeted_status.full_text,user.created_at,user.favourites_count,user.followers_count,user.friends_count,user.id_str,user.location,user.verified,retweet_count,is_troll,sentiment_tag,sentiment_sc
0,2021-01-24 11:09:43+00:00,0,el lelo andrés arauz nos ha dicho q ha gestion...,1353298931913019393,es,Sat Jan 23 22:05:45 +0000 2021,El LELO Andrés Arauz nos ha dicho q ha gestion...,2019-07-08 04:50:16+00:00,23792,208,814,1148091911183503360,NaN,False,212,False,Negativo,0.072007
1,2021-01-24 11:11:10+00:00,0,a qué edad se enteraron que jorge glas y andré...,1353299298407096320,es,Sat Jan 23 23:38:46 +0000 2021,A qué edad se enteraron que Jorge Glas y André...,2012-10-09 14:57:55+00:00,1835,88,325,870030324,NaN,False,545,False,Negativo,0.311006


In [11]:
len(df_all_tweets)

40426

## Tokenization and StopWord

In [8]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

In [35]:
def stopword_removal(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = text.lower()
    tokens = tokenizer.tokenize(text)
    return " ".join([i for i in tokens if i not in stopwords_])
    


In [36]:
stopwords_ = stopwords.words('spanish')
STOPWORDS = set(stopwords_)
df_all_tweets['full_text'] = df_all_tweets['full_text'].apply(stopword_removal)

In [37]:
df_tweets_tokens.head(2)

,created_at,favorite_count,full_text,id_str,metadata.iso_language_code,retweeted_status.created_at,retweeted_status.full_text,user.created_at,user.favourites_count,user.followers_count,user.friends_count,user.id_str,user.location,user.verified,retweet_count,is_troll,sentiment_tag,sentiment_sc
0,2021-01-24 11:09:43+00:00,0,lelo andrés arauz dicho q gestionado gobierno ...,1353298931913019393,es,Sat Jan 23 22:05:45 +0000 2021,El LELO Andrés Arauz nos ha dicho q ha gestion...,2019-07-08 04:50:16+00:00,23792,208,814,1148091911183503360,NaN,False,212,False,Negativo,0.072007
1,2021-01-24 11:11:10+00:00,0,edad enteraron jorge glas andrés arauz primos,1353299298407096320,es,Sat Jan 23 23:38:46 +0000 2021,A qué edad se enteraron que Jorge Glas y André...,2012-10-09 14:57:55+00:00,1835,88,325,870030324,NaN,False,545,False,Negativo,0.311006


In [39]:
len(df_tweets_tokens)

40426

## Lemmatization

In [57]:
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma')

2021-02-16 19:38:28 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |

2021-02-16 19:38:28 INFO: Use device: cpu
2021-02-16 19:38:28 INFO: Loading: tokenize
2021-02-16 19:38:28 INFO: Loading: mwt
2021-02-16 19:38:28 INFO: Loading: pos
2021-02-16 19:38:29 INFO: Loading: lemma
2021-02-16 19:38:29 INFO: Done loading processors!


In [62]:
def lemmatization_stanza(text):
    global nlp
    doc = nlp(text)
    return " ".join([word.lemma for sent in doc.sentences for word in sent.words])

In [65]:
#df_all_tweets['full_text'] = df_all_tweets['full_text'].apply(lemmatization_stanza)

In [67]:
df_all_tweets

,created_at,favorite_count,full_text,id_str,metadata.iso_language_code,retweeted_status.created_at,retweeted_status.full_text,user.created_at,user.favourites_count,user.followers_count,user.friends_count,user.id_str,user.location,user.verified,retweet_count,is_troll,sentiment_tag,sentiment_sc
0,2021-01-24 11:09:43+00:00,0,lelo andrés arauz dicho q gestionado gobierno ...,1353298931913019393,es,Sat Jan 23 22:05:45 +0000 2021,El LELO Andrés Arauz nos ha dicho q ha gestion...,2019-07-08 04:50:16+00:00,23792,208,814,1148091911183503360,NaN,False,212,False,Negativo,0.072007
1,2021-01-24 11:11:10+00:00,0,edad enterar jorge glas andrés arauz primo,1353299298407096320,es,Sat Jan 23 23:38:46 +0000 2021,A qué edad se enteraron que Jorge Glas y André...,2012-10-09 14:57:55+00:00,1835,88,325,870030324,NaN,False,545,False,Negativo,0.311006
2,2021-01-24 11:11:47+00:00,0,extraordinario caravana lojo recibir andrés ar...,1353299454103838720,es,Sun Jan 24 00:05:24 +0000 2021,"Con una extraordinaria caravana, #Loja recibió...",2019-12-07 21:33:55+00:00,20154,73,45,1203427383782629376,NaN,False,466,False,Positivo,0.856285
3,2021-01-24 11:12:50+00:00,0,david villamar mildeunar propuesta andrés arau...,1353299717485158400,es,Sun Jan 24 01:30:39 +0000 2021,"David Villamar: ""Los #MilDeUna, propuesta de A...",2013-08-06 16:33:44+00:00,78836,1008,1396,1650738853,Un Mundo donde haya Justicia,False,292,False,Negativo,0.177461
4,2021-01-24 11:14:35+00:00,0,andrés arauz ganador debatepresidencial según ...,1353300160277852160,es,Sat Jan 23 16:41:37 +0000 2021,Andrés Arauz fue el ganador del #DebatePreside...,2019-08-29 23:23:54+00:00,81000,290,437,1167216174972702724,NaN,False,116,False,Negativo,0.188562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21308,2021-02-13 23:57:34+00:00,0,insultar inteligencia mujer descalificada10 20...,1360739927978373121,es,Sat Feb 13 18:50:05 +0000 2021,Insultan nuestra INTELIGENCIA y la mujer desca...,2012-07-09 03:00:38+00:00,29326,482,247,630728618,Quito,False,1,False,Neutral,0.507002
21309,2021-02-13 23:57:39+00:00,0,andre venega londoño alias uriel dado baja 25 ...,1360739948035510273,es,Sat Feb 13 23:56:29 +0000 2021,"Andres Venegas Londoño, alias 'Uriel', fue dad...",2020-03-29 04:32:00+00:00,7246,56,113,1244119802064576519,NaN,False,55,False,Positivo,0.563317
21310,2021-02-13 23:58:43+00:00,0,atento andrés arauz izquierdista prometer regr...,1360740217834110978,es,NaN,NaN,2009-09-30 19:27:05+00:00,91,1057125,359288,78687173,Venezuela,False,0,False,Negativo,0.018383
19113,2021-02-13 23:59:22+00:00,1,mucho gracias apoyo equipo campaña guillermo l...,1360740379847499780,es,NaN,NaN,2011-03-08 01:40:40+00:00,1608,147,386,262437024,"Marbella, España",False,0,False,Positivo,0.740154


# Save to csv

In [68]:
#The data is saved in the folder data preprocessing with "to_train_models.csv"
df_all_tweets.to_csv(data_processed+"to_train_models.csv")